# SVGBox

In [ ]:
from pathlib import Path
import ipywidgets as W, traitlets as T, IPython as I, lxml.etree as E
from wxyz.svg import SVGBox
from wxyz.svg.widget_svg import DEFAULT_ATTR
from wxyz.html import Fullscreen
from wxyz.lab import DockPop
from wxyz.notebooks import Examples

Consider an `ipywidgets.VBox` or `ipywidgets.HBox`.

In [ ]:
sliders = [W.FloatSlider(description=x, orientation='vertical', min=-100, max=100) for x in "xyz"]
box = W.HBox(sliders, description="Woo")

In [ ]:
if __name__ == "__main__":
    I.display.display(DockPop([box], mode="split-right"))

The `wxyz.svg.SVGBox` is like an except that `children` are layed out to fill the extent of elements of an SVG.

## An SVG

First we need an SVG, like [<img src="./example.svg" align="center" style="height:3em; display: inline-box;"/> `example.svg`](./example.svg), authored in [Inkscape](https://inkscape.org).

In [ ]:
example = Path(Examples.__file__).parent / "example.svg"

In [ ]:
svg = SVGBox(
    sliders,
    svg_file=str(example),
    area_widgets=dict(x=0, y=1, z=2),
    layout=dict(width="100%", height="100%", overflow="hidden")
)

box.children = [svg]

## Areas

It contains a number of elements that have a _XML namespaced-attribute_ defined in `SVGBox.area_attr`.

These areas can be replaced with `children`

In [ ]:
area_widgets = W.HTML()
T.dlink((svg, "area_widgets"), (area_widgets, "value"), lambda x: f"area widgets {x}")

In [ ]:
if __name__ == "__main__":
    I.display.display(area_widgets)

## Visibility

The visibility of many parts of the display can be shown:

In [ ]:
visible_areas = W.Combobox(description="Visible", options=['*', 'x', '(x|z)'])
show_svg = W.Checkbox(description="show SVG")
T.dlink((visible_areas, "value"), (svg, "visible_areas"), lambda x: [x])
T.link((svg, "show_svg"), (show_svg, "value"))

In [ ]:
if __name__ == "__main__":
    I.display.display(visible_areas, show_svg)

## Pan and Zoom

[d3-zoom](https://github.com/d3/d3-zoom) is used for pan and zoom behaviors, and can be read/written.

> **Warning** having multiple copies of the same SVG introduces instability due to `id` clashes, etc.

In [ ]:
zoom_lock = W.Checkbox(description="zoom lock"); W.jslink((svg, "zoom_lock"), (zoom_lock, "value"))
zoom_x = W.FloatSlider(description="zoom_x", min=-100, max=100); W.jslink((svg, "zoom_x"), (zoom_x, "value"))
zoom_y = W.FloatSlider(description="zoom_y", min=-100, max=100); W.jslink((svg, "zoom_y"), (zoom_y, "value"))
zoom_k = W.FloatSlider(description="zoom_k", min=0.01, max=3); W.jslink((svg, "zoom_k"), (zoom_k, "value"));
[W.jslink((s, "value"), (svg, f"zoom_{s.description}")) for s in sliders if s.description in "xy"]
[W.jslink((sliders[i], "value"), (svg, f"zoom_{s}")) for i, s in enumerate("xyk")];

In [ ]:
if __name__ == "__main__":
    I.display.display(zoom_lock, zoom_x, zoom_y, zoom_k)

In [ ]:
app = W.VBox([
    W.HBox([show_svg, area_widgets, visible_areas]),
    svg,
    W.HBox([zoom_lock, zoom_x, zoom_y, zoom_k])
])
box.children = [app]

In [ ]:
if __name__ == "__main__":
    with __import__("importnb").Notebook():
        from wxyz.notebooks import Utils
        Utils.maybe_log_widget_counts()